In [1]:
import tensorflow_decision_forests as tfdf

import tensorflow as tf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

import dtreeviz

from matplotlib import pyplot as plt
from IPython import display

2023-04-26 08:16:33.981901: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 08:16:34.018381: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 08:16:34.019514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 08:16:35.375091: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# avoid "Arial font not found warnings"
import logging
logging.getLogger('matplotlib.font_manager').setLevel(level=logging.CRITICAL)

display.set_matplotlib_formats('retina') # generate hires plots

np.random.seed(1234)  # reproducible plots/data for explanatory reasons

/tmp/ipykernel_566/745931794.py:5: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`


In [3]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]



In [4]:
df = pd.read_csv('covid-19_symptoms_dataset.csv', sep=",")

In [5]:
train_ds_pd, test_ds_pd = split_dataset(df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

1785 examples in training, 790 examples for testing.


In [6]:
df

fever  bodyPain  age  runnyNose  diffBreath  infectionProb
0       102         0    9          0          -1              0
1       102         0   10          0           0              1
2       104         0   33          1          -1              0
3       101         1   59          0           1              0
4        99         0   98          0           0              0
...     ...       ...  ...        ...         ...            ...
2570     99         0   90          0           0              1
2571    100         0   53          0          -1              1
2572    101         0   44          1           0              0
2573    102         0   97          0          -1              1
2574    104         1   62          1          -1              1

[2575 rows x 6 columns]

In [7]:
target = 'infectionProb'

classes = list(df[target].unique())

df = df.iloc[:, df.columns != 'age']

df[target] = df[target].map(classes.index)

/tmp/ipykernel_566/1967664576.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
# Split into training and test sets
train_ds_pd, test_ds_pd = split_dataset(df)
print(f"{len(train_ds_pd)} examples in training, {len(test_ds_pd)} examples for testing.")

# Convert to tensorflow data sets
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=target)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=target)

1800 examples in training, 775 examples for testing.


In [9]:
# Specify the model.
model_1 = tfdf.keras.GradientBoostedTreesModel()

# Train the model.
model_1.fit(train_ds, validation_data=test_ds)

Use /tmp/tmp33a_pckh as temporary training directory
Reading training dataset...


[WARNING 23-04-26 08:16:41.8655 CEST gradient_boosted_trees.cc:1797] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-04-26 08:16:41.8656 CEST gradient_boosted_trees.cc:1808] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-04-26 08:16:41.8656 CEST gradient_boosted_trees.cc:1822] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
2023-04-26 08:16:41.919744: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1800]
	 [[{{node Placeholder/_3}}]]


Training dataset read in 0:00:03.834106. Found 1800 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(775, shape=(), dtype=int32)
Validation dataset read in 0:00:00.195933. Found 775 examples.
Training model...


2023-04-26 08:16:45.745989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [775]
	 [[{{node Placeholder/_1}}]]
[INFO 23-04-26 08:16:47.3703 CEST kernel.cc:1242] Loading model from path /tmp/tmp33a_pckh/model/ with prefix b8763fb44e3345a1


Model trained in 0:00:01.864277
Compiling model...


[INFO 23-04-26 08:16:47.8010 CEST abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-04-26 08:16:47.8010 CEST kernel.cc:1074] Use fast generic engine
2023-04-26 08:16:47.820310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1800]
	 [[{{node Placeholder/_3}}]]


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [10]:
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 0s 80ms/step - loss: 0.0000e+00 - accuracy: 0.5006

loss: 0.0000
accuracy: 0.5006


In [11]:
model_1.save("/tmp/my_saved_model")

INFO:tensorflow:Assets written to: /tmp/my_saved_model/assets


INFO:tensorflow:Assets written to: /tmp/my_saved_model/assets


In [12]:
model_1.summary()

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (4):
	bodyPain
	diffBreath
	fever
	runnyNose

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "diffBreath"  0.601562 ################
    2.  "runnyNose"  0.423077 #######
    3.      "fever"  0.363636 #####
    4.   "bodyPain"  0.253968 

Variable Importance: NUM_AS_ROOT:
    1. "diffBreath"  7.000000 ################
    2.  "runnyNose"  4.000000 

Variable Importance: NUM_NODES:
    1.      "fever" 147.000000 ################
    2.   "bodyPain" 77.000000 ######
    3. "diffBreath" 77.000000 ######
    4.  "runnyNose" 24.000000 

Variable Importance: SUM_SCORE:
    1.      "fever" 43.430714 ######

In [13]:
# The input features
model_1.make_inspector().features()

["bodyPain" (1; #1),
 "diffBreath" (1; #2),
 "fever" (1; #3),
 "runnyNose" (1; #4)]

In [14]:
# The feature importances
model_1.make_inspector().variable_importances()

{'SUM_SCORE': [("fever" (1; #3), 43.430713676161815),
  ("diffBreath" (1; #2), 20.853564599376114),
  ("bodyPain" (1; #1), 14.051393459376413),
  ("runnyNose" (1; #4), 10.303829252254218)],
 'INV_MEAN_MIN_DEPTH': [("diffBreath" (1; #2), 0.6015625),
  ("runnyNose" (1; #4), 0.4230769230769231),
  ("fever" (1; #3), 0.3636363636363637),
  ("bodyPain" (1; #1), 0.253968253968254)],
 'NUM_NODES': [("fever" (1; #3), 147.0),
  ("bodyPain" (1; #1), 77.0),
  ("diffBreath" (1; #2), 77.0),
  ("runnyNose" (1; #4), 24.0)],
 'NUM_AS_ROOT': [("diffBreath" (1; #2), 7.0), ("runnyNose" (1; #4), 4.0)]}

In [15]:
# Tell dtreeviz about training data and model
features = [f.name for f in model_1.make_inspector().features()]
viz_cmodel = dtreeviz.model(model_1,
                           tree_index=3,
                           X_train=train_ds_pd[features],
                           y_train=train_ds_pd[target],
                           feature_names=features,
                           target_name=target,
                           class_names=classes)

ValueError: Tree model must be in (DecisionTreeRegressor, DecisionTreeClassifier, xgboost.core.Booster, lightgbm.basic.Booster, pyspark DecisionTreeClassificationModel, pyspark DecisionTreeClassificationModel, tensorflow_decision_forests.keras.RandomForestModel) but you passed a GradientBoostedTreesModel!

In [ ]:
viz_cmodel.view(scale=1.2)

In [ ]:
print(features)

In [ ]:
model_1.save("my_model")

In [ ]:
df.head(1)

df[~'infectionProb']

In [ ]:
test = tfdf.keras.pd_dataframe_to_tf_dataset(df.loc[:, df.columns!='infectionProb'].head(1))
t1 = df['infectionProb'].head(1)

In [ ]:
tes = model_1.predict(test, verbose=0)

In [ ]:
k = []
i = 0
cnt = 0
for t in tes:
    if(t[0]>0.50):
        k.append(1)
        #print("1 - " + str(t1[i]))
        if(t1[i] == 1):
            cnt += 1
    else:
        k.append(0)
        #print("0 - " + str(t1[i]))
        if(t1[i] == 0):
            cnt += 1
    i+=1
print(cnt/len(t1))

In [ ]:
print(df.head(1))

In [ ]:
t2_dic = {'fever':[103], 'bodyPain': [0], 'runnyNose':[1], 'diffBreath':[-1]}
t2_df = pd.DataFrame.from_dict(t2_dic)

In [ ]:
d2_tfdf = tfdf.keras.pd_dataframe_to_tf_dataset(t2_df)

In [ ]:
model_1.predict(d2_tfdf, verbose=0)